# Paso C - Creación de dataset (train / inferencia)

Este notebook construye el dataset en formato **wide** (una fila por contrato por fecha de corte) a partir de los datos en `data/interim/`.

- **Modo train**: varias fechas de corte con inspecciones; se une con target `is_fraud`.
- **Modo inferencia**: una fecha de corte; se genera dataset sin target para scoring.

Toda la lógica está en **`src.data.make_dataset`**.

In [1]:
import os
import sys
import glob
import re
import pandas as pd
from tqdm import tqdm

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

%load_ext autoreload
%autoreload 2

In [2]:
from src.data.make_dataset import (
    get_date_range_for_cutoff,
    create_train_dataset,
    create_inference_dataset,
)

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

---
## Configuración

Parámetros que ingresa el usuario (o se dejan por defecto).

In [4]:
INTERIM_DIR = "../data/interim"
PROCESSED_DIR = "../data/processed"

# Modo: 'train' o 'inference'
MODE = "inference"

# Fecha de corte (ej. "2024-08-01"). En train se usan todas las fechas <= CUTOFF con inspecciones.
# En inference es la única fecha para la cual se arma el dataset.
CUTOFF = "2025-08-01"

# Ventana de meses de consumo hacia atrás
CANT_PERIODOS = 12

# Solo para inferencia: lista de contratos a scorear (None = todos los que tengan consumo en la ventana)
CONTRATOS_LIST = None

# Rango de meses que se cargarán desde interim (solo estos se levantan en memoria)
START_LOAD, END_LOAD = get_date_range_for_cutoff(CUTOFF, CANT_PERIODOS)
print(f"Rango a cargar desde interim: {START_LOAD.strftime('%Y-%m-%d')} a {END_LOAD.strftime('%Y-%m-%d')} ({CANT_PERIODOS + 1} meses)")

Rango a cargar desde interim: 2024-08-01 a 2025-08-01 (13 meses)


---
## Ejecutar creación de dataset

Según MODE se ejecuta train o inferencia.

In [5]:
%%time
if MODE == "train":
    df_result = create_train_dataset(
        INTERIM_DIR, PROCESSED_DIR,
        cant_periodos=CANT_PERIODOS,
        cutoff_max=CUTOFF
    )
else:
    df_result = create_inference_dataset(
        INTERIM_DIR, PROCESSED_DIR,
        cutoff=CUTOFF,
        cant_periodos=CANT_PERIODOS,
        contratos_list=CONTRATOS_LIST
    )

*** Feature extraction started ***


d:\2024\BID\Aguas\Empresa2-Queretaro\proyecto\queretaro_poc\src\data\make_dataset.py:28: UserWarning: Using default sampling frequency set in configuration file.
  df_result = tsfel.time_series_features_extractor(cfg, df[cols].values.tolist(), verbose=1, n_jobs=-1)



*** Feature extraction finished ***
*** Feature extraction started ***


d:\2024\BID\Aguas\Empresa2-Queretaro\proyecto\queretaro_poc\src\data\make_dataset.py:28: UserWarning: Using default sampling frequency set in configuration file.
  df_result = tsfel.time_series_features_extractor(cfg, df[cols].values.tolist(), verbose=1, n_jobs=-1)



*** Feature extraction finished ***
*** Feature extraction started ***


d:\2024\BID\Aguas\Empresa2-Queretaro\proyecto\queretaro_poc\src\data\make_dataset.py:28: UserWarning: Using default sampling frequency set in configuration file.
  df_result = tsfel.time_series_features_extractor(cfg, df[cols].values.tolist(), verbose=1, n_jobs=-1)



*** Feature extraction finished ***
[OK] Guardado: ../data/processed\inference\cutoff=2025-08-01\inference_wide.parquet (11019 filas)
CPU times: total: 6min 47s
Wall time: 12min 29s


---
## Verificación

Revisión rápida del dataset generado.

In [6]:
if df_result is not None:
    print("Shape:", df_result.shape)
    print("Columnas:", df_result.columns.tolist()[:15], "...")
    if MODE == "train" and "is_fraud" in df_result.columns:
        print("Distribución is_fraud:")
        print(df_result["is_fraud"].value_counts(normalize=True))
    print("\nPrimeras filas:")
    display(df_result.head(3))

Shape: (11019, 225)
Columnas: ['contrato', '12_anterior', '11_anterior', '10_anterior', '9_anterior', '8_anterior', '7_anterior', '6_anterior', '5_anterior', '4_anterior', '3_anterior', '2_anterior', '1_anterior', 'date_fizcalizacion', 'cant_tipo_serv'] ...

Primeras filas:


,contrato,12_anterior,11_anterior,10_anterior,9_anterior,8_anterior,7_anterior,6_anterior,5_anterior,4_anterior,...,mean_12,cant_ceros_12,max_cant_ceros_seg_12,slope_12,min_cons12,max_cons12,std_cons12,var_cons12,skew_cons12,kurt_cons12
0,271971,0.000000,0.0,15.0,0.0,0.0,0.0,15.0,15.0,15.0,...,8.750000,5,3,1.520979,0.000000,15.0,7.723930,59.659091,-0.388403,-2.262857
1,271998,6.333333,19.0,17.0,16.0,17.0,22.0,15.0,15.0,15.0,...,15.611111,0,0,0.095571,6.333333,22.0,3.626038,13.148148,-1.132323,4.366733
2,272044,2.000000,1.0,2.0,1.0,6.0,2.0,1.0,1.0,1.0,...,1.666667,0,0,-0.118881,1.000000,6.0,1.435481,2.060606,2.917653,9.112111
